<a href="https://colab.research.google.com/github/sat2567/CNN/blob/main/Copy_of_SATYA_PRAKASH_MISHRA_l1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1 MB 1.6 kB/s 
     |████████████████████████████████| 8.8 MB 20.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.2.0+cu92 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.2.0+cu92 which is incompatible.
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:

%tensorflow_version 2.x  # this line is not required unless you are in a notebook
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import torch
from tensorflow.keras import layers
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.io

import math
import sklearn
import sklearn.datasets


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


# Section 2: Classification with Convolutional Neural Networks

**Total for this subsection: 24 points**

---

In this section we will perform a binary classification task of identifying whether an image contains a Burger or Vada pav. The dataset can be downloaded from Kaggle at (https://www.kaggle.com/meemr5/vadapav). The aim of this section is to provide a warmup of how to find optimal hyperparameters for a given classification task.

## To do

---

1. Download the pretrained ResNet50 model from the Keras or TorchVision library. **(1 point)**
2. Modify the final fully connected layer to accomodate 2 classes. **(1 point)**
3. Display the modified model architecture **(1 point)**
4. The dataset has the following directory structure:

<pre>
<b>Root directory</b>
|__ <b>burger</b> : [all burger images]
|__ <b>vadapav</b>: [all vadapav images]
</pre>

Modify the structure as 

<pre>
<b>Root directory</b>
|__ <b>train</b>
    |______ <b>burger</b>: [80% burger images]
    |______ <b>vadapav</b>: [80% vadapav images]
|__ <b>validation</b>
    |______ <b>burger</b>: [10% burger images]
    |______ <b>vadapav</b>: [10% vadapav images]
|__ <b>test</b>
    |______ <b>burger</b>: [10% burger images]
    |______ <b>vadapav</b>: [10% vadapav images]
</pre>
**(3 x 2 = 6 points)**
5. Prepare the data loaders/generators for the train, validation and test sets from which mini batches will be sampled. You may use an optimal batch size as per your available resources.**(3 x 1 = 3 points)**
6. Create an Adam Optimizer instance whose learning rate is variable **(2 points)**
7. Compile your model with a suitable loss function for the binary classification task **(1 point)**
8. Perform grid search over a set of learning rates from $10^{-5}$ till $10^{-1}$ in steps of 0.1 to find the optimal learning rate at which the validation accuracy peaks. Plot a line graph to show the trend of training and validation set accuracies with the increase of learning rate. Save the model learned at the optimal learning rate. **(2 points for grid search + 2 points for generating line plot + 2 points for saving the optimal model = 6 points)**
9. Load the optimal model saved. Determine and print its accuracy on the test set. Compare the learned classifier with the random classifier. **(1 + 1 + 1 = 3 points)**



In [ ]:
# All CNN experiments start here
!pip install resnet


  Created wheel for resnet: filename=resnet-0.1-py3-none-any.whl size=10042 sha256=cd9b776be49783404055b8d8a9582bbd56a8b1127e9d7eceb4dded6e4255c16b
  Stored in directory: /root/.cache/pip/wheels/38/23/58/6e83e34ced7d4772b88e2e86305f94a1ca497bd87937f5e7ec
Successfully built resnet


In [ ]:
from keras.applications.resnet import ResNet50
import torchvision
from torchvision import datasets, models, transforms
import torch
import torch.nn as nn
import torch.optim as optim

from torch.optim import lr_scheduler


In [ ]:
model1 = models.resnet50(pretrained=True)
print(model1)
num_ftrs = model1.fc.in_features
print(num_ftrs)
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model1.fc=nn.Linear(num_ftrs,3)
'''model.fc =nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 2),
    nn.LogSoftmax(dim=1) )'''

print(model1.fc)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 62.6MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import os
import sys

In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
import opendatasets as od

dataset_url = 'https://www.kaggle.com/meemr5/vadapav'
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gokulnandantm
Your Kaggle Key: ··········


100%|██████████| 295M/295M [00:04<00:00, 70.1MB/s]


In [ ]:
import os
DATA_DIR = './vadapav/VadaPav'
print(os.listdir(DATA_DIR))

['burger', 'vada pav']


In [ ]:
pip install split-folders

In [ ]:
import splitfolders

In [ ]:
splitfolders.ratio(input="/content/vadapav/VadaPav", output="/content/vadapav/VadaPav",  ratio=(.8, .1, .1), group_prefix=None) # default valu

Copying files: 1161 files [00:01, 871.61 files/s] 


In [ ]:
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T

In [ ]:
image_size = 64
batch_size = 128
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

In [ ]:
image_transforms = {
    '/content/vadapav/Vada_pav/train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}


In [ ]:

image_data = ImageFolder('/content/vadapav/VadaPav', transform=T.Compose([
    T.Resize(image_size),
    T.CenterCrop(image_size),
    T.ToTensor(),
    T.Normalize(*stats)]))

image_dl = DataLoader(image_data, batch_size, shuffle=True, num_workers=2, pin_memory=True)
type(image_data)

torchvision.datasets.folder.ImageFolder

In [ ]:
rmdir/content/vadapav/VadaPav/test/.ipynb_checkpoints


rmdir: failed to remove '/content/vadapav/VadaPav/test/.ipynb_checkpoints': No such file or directory


In [ ]:
len('/content/vadapav/VadaPav/test')
num_classes = len(os.listdir("/content/vadapav/VadaPav/val"))+1
os.listdir("/content/vadapav/VadaPav/test")

['burger', 'vada pav']

In [ ]:
train_size = int(0.8 * len(image_data))
validation_size=int(0.1*len(image_data))
test_size = len(image_data)-(train_size+validation_size)
train_dataset , validation_dataset, test_dataset = torch.utils.data.random_split(image_data, [train_size,validation_size, test_size])

In [ ]:
training_dl = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=2, pin_memory=True)
validation = DataLoader(validation_dataset, batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_dl = DataLoader(test_dataset, batch_size, shuffle=True, num_workers=2, pin_memory=True)


In [ ]:
dataset="/content/vadapav/VadaPav"

In [ ]:
import torch.optim as optim

In [ ]:
loss_func = nn.NLLLoss()
optimizer = optim.Adam(model1.parameters())

In [ ]:
import time
import torch, torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt
import os

from PIL import Image

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_and_validate(model, loss_criterion, optimizer, epochs=25):
    '''
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param loss_criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)
  
    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    '''
    
    start = time.time()
    history = []
    best_loss = 100000.0
    best_epoch = None

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in enumerate(training_dl):

            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs,labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
            #print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

        
        # Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(validation):
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)

                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
        if valid_loss < best_loss:
            best_loss = valid_loss
            best_epoch = epoch

        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_size 
        avg_train_acc = train_acc/train_size

        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss/validation_size 
        avg_valid_acc = valid_acc/validation_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss - {:.4f}, Accuracy - {:.4f}%, \n\t\tValidation : Loss - {:.4f}, Accuracy - {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
        
        # Save if the model has best accuracy till now
        torch.save(model, dataset+'_model_'+str(epoch)+'.pt')
            
    return model, history, best_epoch

In [ ]:
num_epochs = 5
trained_model, history, best_epoch = train_and_validate(model1, loss_func, optimizer, num_epochs)

torch.save(history, dataset+'_history.pt')

Epoch: 1/5


ValueError: ignored

In [ ]:
data1 = data
data1["Target"]=Target